## Fit

In [ ]:
def sin_fit(x, y, resample=501):
	# фитует результаты измерений экспонентой
	def model(x, p):
		phase = p[0]
		freq = p[1]
		A = np.reshape(np.asarray(p[2:]),(len(p[2:]), 1))
		return A*np.cos(phase+x*freq*2*np.pi)
	
	cost = lambda p: (np.abs(model(x, p)-y)**2).ravel()
	
	means = np.reshape(np.mean(y, axis=1), (np.asarray(y).shape[0], 1))
	y = y-means
	
	ft = np.fft.fft(y-np.reshape(np.mean(y, axis=1), (y.shape[0], 1)), axis=1)/len(x)
	f = np.fft.fftfreq(len(x), x[1]-x[0])
	domega = (f[1]-f[0])*2*np.pi
	
	fR_id = np.argmax(np.sum(np.abs(ft)**2, axis=0))
	fR_id_conj = len(f)-fR_id
	if fR_id_conj > fR_id:
		tmp = fR_id_conj
		fR_id_conj = fR_id
		fR_id = tmp
	
	fR = np.abs((f[fR_id]))
	
	c = np.real(np.sum(ft[:,fR_id], axis=0))
	s = np.imag(np.sum(ft[:,fR_id], axis=0))
	phase = np.arctan2(s, c)
	#x0 = np.sqrt(np.mean(np.abs(ft[:,fR_id])**2)/np.mean(np.abs((ft[:,fR_id-1]+ft[:,fR_id+1])/2)**2)-1)/domega/2
	#print (np.abs(ft[:,fR_id])**2, np.abs((ft[:,fR_id-1]+ft[:,fR_id+1])/2)**2)
	
	A = np.sqrt(np.abs(ft[:,fR_id])**2+np.abs(ft[:,fR_id_conj])**2)
	p0 = [phase, fR]+A.tolist()
	
	from scipy.optimize import leastsq
	fitresults_all = []
	for random_id in range(100):
		random_samples = np.random.choice(len(x), int(len(x)*0.7))
		cost = lambda p: (np.abs(model(x[random_samples], p)-y[:,random_samples])**2).ravel()
		
		fitresults = leastsq (cost, p0)
		fitresults_all.append(fitresults[0])
	fitted_curve = model(x, fitresults[0])
	
	resampled_x = np.linspace(np.min(x), np.max(x), resample)
	resampled_y = model(resampled_x, fitresults[0])+means
	
	#for i in range(4):
	#	plt.figure(i)
	#	plt.plot(x, y[i,:], label='data')
	#	plt.plot(x, model(x, p0)[i,:], label='initial')
	#	plt.plot(x, model(x, fitresults[0])[i,:], label='fitted')
	#	plt.legend()
	
	return fitresults[0], fitted_curve+means, fitresults_all, resampled_x, resampled_y    

In [ ]:
from scipy.optimize import curve_fit

def set_ex_amp(amp,num_pulses):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
                [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
    
def func(x,a,b,c):
    return a*x**2+c*x+b

def func_for_fit_cal(x,a,b,c):
    return a*x**(5/3)+b*x**(4/3)+c

def fit_cal(data):
    x = [data[i][0] for i in range(len(data))]
    y = [data[i][1] for i in range(len(data))]
    popt = curve_fit(func_for_fit_cal,x,y)[0]
    angles = np.linspace(x[np.argmin(x)],x[np.argmax(x)],1000)
    amps = func_for_fit_cal(angles,popt[0],popt[1],popt[2])
    return angles,amps,[popt[0],popt[1],popt[2]]

def find_max(x,y):
    popt=curve_fit(func,x,y)[0]
    n1 = np.linspace(x[np.argmin(x)],x[np.argmax(x)],100)
    n2 = func(n1,popt[0],popt[1],popt[2])
    max_amp=n1[np.argmax(n2)]
    return max_amp

def first_sweep(num_pulses,ex_amps):# in order to find a period between first two peaks
    set_ex_first = lambda amp: set_ex_amp(amp,num_pulses)
    print('ya tut first sweep')
    measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_first, 'Rabi pulse amplitude'),
                                      filename=None,save=False)
    x = measurement['S21+'][1][0]
    y = (np.real(measurement['S21-'][2]),np.imag(measurement['S21-'][2]))
    fit= sin_fit(x,y)[0]
    x_max_0,x_min_0=(np.pi-fit[0])/(2*np.pi*fit[1]),(np.pi*4/2-fit[0])/(2*np.pi*fit[1])
    cut=np.abs((x_max_0-x_min_0))/4
    ex_amps=np.linspace(x_max_0-cut,x_max_0+cut,10)
    T0=1/fit[1]
    return T0,cut,ex_amps

def set_amps(num_pulses,ex_amps_first):
    T0,cut,ex_amps = first_sweep(num_pulses,ex_amps_first)
    print(T0,cut,ex_amps)
    amp_for_angle=[]
    k=0
    set_ex_next = lambda amp: set_ex_amp(amp,num_pulses)
    for i in range(int(num_pulses/2)):
        measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_next, 'Rabi pulse amplitude'),
                                      filename=None,save=False)
        x=measurement['S21-'][1][0]
        y=np.angle(measurement['S21-'][2])
        max_amp=find_max(x,y)
        amp_for_angle.append([(1+2*i),max_amp])
        if k>0:
            T=amp_for_angle[i][1]-amp_for_angle[i-1][1]
            print(T)
            ex_amps=np.linspace(max_amp+T-cut,max_amp+T+cut,10)
        else:
            ex_amps=np.linspace(max_amp+T0-cut,max_amp+T0+cut,10)
            k=1
        print('angle: {}pi, amplitude: {}'.format((1+2*i)/num_pulses,max_amp))
    return amp_for_angle

In [ ]:
num_pulses = 12
ex_amps_first=np.linspace(0.03,0.6,81)
amps=set_amps(num_pulses,ex_amps_first)
angles_curve = fit_cal(amps)

In [ ]:
x=angles_curve[0]
y=angles_curve[1]
params_of_fit=angles_curve[3]
print(params)
plt.plot(x,y,"-")
plt.plot([amps[0][0],amps[(len(amps)-1)][0]],[amps[0][1],amps[(len(amps)-1)][1]])
plt.plot([amps[i][0] for i in range(len(amps))],[amps[i][1] for i in range(len(amps))],'o')
plt.show()